<a href="https://colab.research.google.com/github/Espebaum/Gemma2b-it-Write-My-Paper/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U bitsandbytes # 양자화를 위한 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9

In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [4]:
import json
import pandas as pd

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
test_path = '/content/drive/MyDrive/Colab_Notebooks/Gemma2/paper_validation/paper_validation.json'

with open(test_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터를 담을 리스트 초기화
documents = []

# 2. 데이터 파싱
for item in data[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_entire', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_entire', [])])
    }
    documents.append(doc)

for item in data[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_section', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_section', [])])
    }
    documents.append(doc)

# 3. 데이터프레임으로 변환
df_test = pd.DataFrame(documents)
df_test.head()

,category,title,summary,original
0,사회과학,몰입(Flow) 이론을 적용한 진로상담모형,Csikszentmihalyi의 몰입이론을 적용한 진로상담의 모형을 제시하는 것이다...,몰입이론은 원래 창조적인 활동을 하는 사람들이 자신의 일에 몰입할 때 경험하는 현상...
1,사회과학,지역사회 청소년 상담기관의 실제와 연계체제 활성화 방안에 관한 현장 연구: 안양지역...,지역사회 청소년 상담기관에서 실제로 상담업무를 담당하는 실무자들과의 면접조사를 통해...,본 연구는 지역사회 청소년 상담기관에서 실제로 상담업무를 담당하는 실무자들과의 면접...
2,사회과학,아동의 자살 생각에 영향을 미치는 가족 위험요인,아동의 자살생각에 영향을 미치는 가족 위험요인을 알아보고자 하였다.\n집단간 가족특...,본 연구에서는 아동의 자살생각에 영향을 미치는 가족 위험요인을 알아보고자 하였다. ...
3,사회과학,자기애적 성격성향과 청소년 비행성향간의 관계,"자기애적 성격성향과 청소년 비행성향 간의 관계를 알아보고, 청소년 비행성향에 영향을...","현대사회는 자기애적 현상이 문화적 흐름으로 강하게 자리잡고 있고, 우리나라에도 이 ..."
4,사회과학,청소년의 음란물 접촉과 잘못된 강간통념 수용간의 매개변인과 조정변인의 역할,청소년들의 음란물 접촉은 날로 증가하지만 이를 효과적으로 차단하기 힘든 현 상황에서...,"본 연구에서는 청소년들의 음란물 접촉은 날로 증가하지만, 이를 효과적으로 차단하기 ..."


In [7]:
dataset_test = Dataset.from_pandas(df_test)
doc = dataset_test[115]['summary']
origin = dataset_test[115]['original']
doc

' 한ㆍ중ㆍ일 3국의 정치적 태도와 사회경제체제를 ‘세계가치관조사’ 자료를 이용하여 국가 간 비교분석하였다. 중국의 아시아적 가치 지향성이 가장 높고 일본이 가장 낮다. 서구적 민주주의 지향성은 일본이 가장 높고 한국이 그 다음이며 중국이 가장 낮다. 한ㆍ중ㆍ일 3국의 정치적 태도에서의 차이는 사회구조적 변동 경험을 반영하는 것이다. '

In [10]:
print(dataset_test.shape)
dataset_test[115]

(36000, 4)


{'category': '사회과학',
 'title': '한ㆍ중ㆍ일 3국의 정치적 태도 비교 연구: 민주주의와 사회경제체제에 대한 태도를 중심으로',
 'summary': ' 한ㆍ중ㆍ일 3국의 정치적 태도와 사회경제체제를 ‘세계가치관조사’ 자료를 이용하여 국가 간 비교분석하였다. 중국의 아시아적 가치 지향성이 가장 높고 일본이 가장 낮다. 서구적 민주주의 지향성은 일본이 가장 높고 한국이 그 다음이며 중국이 가장 낮다. 한ㆍ중ㆍ일 3국의 정치적 태도에서의 차이는 사회구조적 변동 경험을 반영하는 것이다. ',
 'original': '이 연구는 한ㆍ중ㆍ일 3국의 정치적 태도와 사회경제체제를 ‘세계가치관조사’ 자료를 이용하여 국가 간 비교분석을 시도하였다. 중국의 아시아적 가치 지향성이 가장 높고 일본이 가장 낮으며 한국은 그 중간이다. 서구적 민주주의 지향성은 일본이 가장 높고 한국이 그 다음이며 중국이 가장 낮다. 중국인이 한국인과 일본인보다 상대적으로 자본주의 지향성이 높고 보수적인데 반해, 일본인은 개혁 지향적이고 한국인은 상대적으로 혁명 지향적이다. 한ㆍ중ㆍ일 3국의 정치적 태도에서의 이러한 차이는 그동안의 사회구조적 변동 경험을 반영하는 것이다. 중국에서 민주적 정치문화가 확산되고 있기는 하지만 중국의 정치지도자들이 민주주의의 이상을 추구하기보다는 실용적이고 기능적으로 민주주의에 접근하고 있는 한 중국의 정치적 민주화는 급격하게 단시일에 실현되기보다는 좀 더 시간이 필요할 것이다.'}

In [11]:
FINETUNED_MODEL = "gyopark/gemma-2-2b-it-WriteMyPaper"

model = AutoModelForCausalLM.from_pretrained(FINETUNED_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [12]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
) # gemma에서 제공하는 파이프라인이라고 한다.

In [13]:
messages = [
    {
        "role": "user",
        "content": "다음 요약된 글을 한 문단으로 길게 써주세요.:\n\n{}".format(doc)
    }
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 요약된 글을 한 문단으로 길게 써주세요.:\n\n 한ㆍ중ㆍ일 3국의 정치적 태도와 사회경제체제를 ‘세계가치관조사’ 자료를 이용하여 국가 간 비교분석하였다. 중국의 아시아적 가치 지향성이 가장 높고 일본이 가장 낮다. 서구적 민주주의 지향성은 일본이 가장 높고 한국이 그 다음이며 중국이 가장 낮다. 한ㆍ중ㆍ일 3국의 정치적 태도에서의 차이는 사회구조적 변동 경험을 반영하는 것이다.<end_of_turn>\n<start_of_turn>model\n'

In [16]:
outputs = pipe(
    prompt,
    do_sample=True,
    add_special_tokens=True,
    max_new_tokens=1024
)

outputs[0]["generated_text"][len(prompt):]

'한ㆍ중ㆍ일 3국 간 국가 간 비교분석을 위해 ‘세계가치관조사’(World Values Study)1)에 사용된 자료를 활용하였다. 39개의 각국의 정치적 태도는 가치관조사 자료에 제시된 문항을 수집하여 9개의 선형요인으로 구성하였고 세 가지 요인은 모두 1점부터 5점까지의 평균정도로 측정되었다. ‘세계가치관조사’의 데이터 분석에서는 각 국가 간 정치적 태도의 차이가 어느 정도인가를 측정하는 등의 분석을 수행하였다. 중국, 일본 및 한국 3국의 아시아적 가치 지향성이 가장 높다. 유럽과 북미 및 중동 지역이 가장 낮다. 서구적 민주주의 지향성은 한국, 중국, 일본 순으로 일본이 가장 높고 미국, 남아시아 및 유럽 등이 가장 낮다. 외국인 인구를 백분율로 표현하였을 때 일본이 1분의 8 미만인 반면, 한국은 2.3분의 1, 중국은 1.8분의 1이다. 3국 간 중동, 북아시아 및 중동 지역이 다른 지역에 비해 낮은 농촌지향성을 지닐 것이다. 3국 간 유엔인구사에서 2000년 인구에 대한 중국인, 일본인, 한국인이 높은 비중을 기입하고 있다. 미국은 1980년에 인구의 6.2%, 1990년에는 7.4%, 일본은 1980년에 인구의 3.1%, 1990년에는 2.8%에 해당한다. 이들 3국인 2000년도에 중동과 북아시아에 사는 유엔인구의 상대 비중을 포함하면 중동은 여성의 비중이 높고 북아시아는 남성의 비중이 높다. 한국의 여성의 비중이 가장 높고 중국의 남성 비중이 가장 높다. 사회구조적 변동 경험에 의해 국가 간 정치적 태도의 차이가 나타나고 있다. 이들은 동아시아 내의 정치 경험과 그 역사적 경험에 의한 차이를 중심으로 한다.'

In [17]:
origin # 원본

'이 연구는 한ㆍ중ㆍ일 3국의 정치적 태도와 사회경제체제를 ‘세계가치관조사’ 자료를 이용하여 국가 간 비교분석을 시도하였다. 중국의 아시아적 가치 지향성이 가장 높고 일본이 가장 낮으며 한국은 그 중간이다. 서구적 민주주의 지향성은 일본이 가장 높고 한국이 그 다음이며 중국이 가장 낮다. 중국인이 한국인과 일본인보다 상대적으로 자본주의 지향성이 높고 보수적인데 반해, 일본인은 개혁 지향적이고 한국인은 상대적으로 혁명 지향적이다. 한ㆍ중ㆍ일 3국의 정치적 태도에서의 이러한 차이는 그동안의 사회구조적 변동 경험을 반영하는 것이다. 중국에서 민주적 정치문화가 확산되고 있기는 하지만 중국의 정치지도자들이 민주주의의 이상을 추구하기보다는 실용적이고 기능적으로 민주주의에 접근하고 있는 한 중국의 정치적 민주화는 급격하게 단시일에 실현되기보다는 좀 더 시간이 필요할 것이다.'